In [1]:
import numpy as np
import pandas as pd
import os
import re
import difflib
from collections import Counter
from metaphone import doublemetaphone

ROOT = '/home/thabib/database_records_linkage'

In [8]:
s1_s2_commonids = pd.read_csv(os.path.join(ROOT, 'data', 's1_s2_commonids.tsv'), sep='\t')
s1_cstr = pd.read_csv(os.path.join(ROOT,'data', 'source1_cstr.csv'))
s2_nond_cstr = pd.read_csv(os.path.join(ROOT, 'data', 'source2_nonanids_nod_cstr.csv'))

In [9]:
s1_cstr

,id,name,street_number,street_type,street_name,address_line2,postal_code,city
0,542012000015,societe des sucreries du marquenterre,NaN,rue,de la fontaine,NaN,80120,rue
1,542012000031,societe des sucreries du marquenterre,NaN,rte,nationale,NaN,62140,marconnelle
2,545011900028,le grand cafe et gassendi,NaN,NaN,saint christophe,route de marseille,4000,digne les bains
3,552017600016,hernas cartonnage,50.0,rue,pasteur,NaN,80210,feuquieres en vimeu
4,552017600032,hernas cartonnage,NaN,NaN,zone des hayettes,NaN,80210,chepy
...,...,...,...,...,...,...,...,...
1826164,90771119600010,bionormande,NaN,rue,de la grande flandre,lot 1 7 a,76270,neufchatel en bray
1826165,90771120400012,ahela bat,18.0,rue,saint clement,NaN,93200,saint denis
1826166,90771121200015,deco batiment,7.0,rue,auguste blanqui,NaN,93200,saint denis
1826167,90760906900014,chateaux merveilles,15.0,rue,de milan,NaN,75009,paris 9


In [12]:
unique_street_types = np.unique(s1_cstr['street_type'].dropna())

In [13]:
s2_nond_cstr

,Unnamed: 0,address,website,id,name
0,1,50 rte bage 01380 saint cyr sur menthon,NaN,84122632700010,sarl landrix freres
1,2,120 r menthon 01380 saint cyr sur menthon,NaN,79134582000025,marion kwasniewski
2,4,350 rte loeze 01380 saint cyr sur menthon,NaN,43416176600010,manigand stephane
3,5,41 r centre 01380 saint cyr sur menthon,http://www.infirmiere-frederique-thevenard.fr,48854644100012,frederique thevenard
4,6,116 chem cochatieres 01380 saint cyr sur menthon,NaN,41811188600028,garnier xavier
...,...,...,...,...,...
2271357,3530296,2 av chateau 83150 bandol,NaN,30130663500097,carlot jean francois
2271358,3530297,NaN,NaN,84978102600018,jacoby o
2271359,3530299,5 r jardins 83150 bandol,http://aquabike-en-mer.com,79518496900025,calvet richard
2271360,3530300,54 traverse laboureur 83150 bandol,http://dolcefarnientebandol.com,53925863200014,dolce far niente


# Testing Double Metaphone on name_x, name_y similarity matching

In [10]:
s1_s2_commonids.iloc[13]

Unnamed: 0                                                  13
id                                                565014800033
name_x                                 SCIC VALDURANCE HABITAT
street_number                                              2.0
street_type                                                RUE
street_name                            DU DOCTEUR SIMON PIETRI
address_line2                                              NaN
postal_code                                               4000
city                                           DIGNE-LES-BAINS
address          Bp 169 2 r Doct Piétri, 04000 DIGNE LES BAINS
website                                                    NaN
name_y                                 Valdurance Habitat SCIC
Name: 13, dtype: object

In [135]:
x_ = 'SCIC VALDURANCE HABITAT'.split(' ')
x_.sort()
x_ = ' '.join(x_)
print(doublemetaphone(x_))

y_ = 'Valdurance Habitat SCIC'.split(' ')
y_.sort()
y_ = ' '.join(y_)
print(doublemetaphone(y_))

('HPTTSKFLTRNS', '')
('HPTTSKFLTRNS', '')


In [137]:
y_

'Habitat SCIC Valdurance'

In [124]:
doublemetaphone('SCIC VALDURANCE HABITAT')

('SKFLTRNSPTT', '')

In [125]:
doublemetaphone('Valdurance Habitat SCIC')

('FLTRNSPTTSK', '')

In [45]:
s1_s2_commonids.iloc[770311]['name_x']

"SOCIETE D'EXERCICE LIBERALE A RESPONSABILITE LIMITEE DE CHIRURGIENS DENTISTES PATRICK SEBAG"

In [54]:
x = s1_s2_commonids.iloc[770311]['name_x'].split(' ')
x.sort()
x = ' '.join(x)
x

"A CHIRURGIENS D'EXERCICE DE DENTISTES LIBERALE LIMITEE PATRICK RESPONSABILITE SEBAG SOCIETE"

In [46]:
s1_s2_commonids.iloc[770311]['name_y']

'Chirurgiens Dentistes Patr'

In [58]:
y = s1_s2_commonids.iloc[770311]['name_y'].split(' ')
y.sort()
y = ' '.join(y)
y

'Chirurgiens Dentistes Patr'

In [59]:
doublemetaphone(x)

('AXRRJNSTTKSRSSTTNTSTSLPRLLMTPTRKRSPNSPLTSPKSST',
 'AKRRKNSTTKSRSSTTNTSTSLPRLLMTPTRKRSPNSPLTSPKSXT')

In [63]:
doublemetaphone(y)

('XRRJNSTNTSTSPTR', 'XRRKNSTNTSTSPTR')

In [145]:
print(get_overlap('THR', ''))

['']


In [132]:
''.join(get_overlap(doublemetaphone('Valdurance Habitat SCIC')[0], doublemetaphone('SCIC VALDURANCE HABITAT')[0]))

'FLTRNSPTT'

In [95]:
doublemetaphone('JANELI')

('JNL', 'ANL')

In [96]:
doublemetaphone('Intermarché Super')

('ANTRMRXSPR', 'ANTRMRKSPR')

# My Algorithm for Similarity MAtching for Strings

In [43]:
def get_overlap(s1, s2):
    # From: https://stackoverflow.com/questions/14128763/how-to-find-the-overlap-between-2-sequences-and-return-it
    s = difflib.SequenceMatcher(None, s1, s2)
    list_t = s.get_matching_blocks()
    #print(list_t)
    best_overlaps = []
    for i in range(len(list_t)):
        best_overlaps.append(s1[list_t[i][0]:list_t[i][0]+list_t[i][2]])
    return best_overlaps

def get_match_score(s1_name, s2_name):
    
    s1_ = s1_name.split(' ')
    s1_.sort()
    s1_ = ' '.join(s1_)
    s1_name_dmp = doublemetaphone(s1_)
    #print(s1_name_dmp)
    s2_ = s2_name.split(' ')
    s2_.sort()
    s2_ = ' '.join(s2_)
    s2_name_dmp = doublemetaphone(s2_)
    #print(s2_name_dmp)
    
    t1_t1_overlap = ''.join(get_overlap(s1_name_dmp[0], s2_name_dmp[0]))
    if s1_name_dmp[1] == '' and s2_name_dmp[1] == '':
        t2_t2_overlap = [1]
    else:
        t2_t2_overlap = ''.join(get_overlap(s1_name_dmp[1], s2_name_dmp[1]))
    t1_t2_overlap = ''.join(get_overlap(s1_name_dmp[0], s2_name_dmp[1]))
    t2_t1_overlap = ''.join(get_overlap(s1_name_dmp[1], s2_name_dmp[0]))
    
    t1_t1_score = len(t1_t1_overlap)/max(len(s1_name_dmp[0]), len(s2_name_dmp[0]), 1)
    t2_t2_score = len(t2_t2_overlap)/max(len(s1_name_dmp[1]), len(s2_name_dmp[1]), 1)
    t1_t2_score = len(t1_t2_overlap)/max(len(s1_name_dmp[0]), len(s2_name_dmp[1]), 1)
    t2_t1_score = len(t2_t1_overlap)/max(len(s1_name_dmp[1]), len(s2_name_dmp[0]), 1)
    
    match_score = t1_t1_score + 0.5*t2_t2_score + 0.25*(t1_t2_score + t2_t1_score)
    
    return match_score

In [45]:
get_match_score('av Gare, ANNOT','GARE DES CHEMINS DE FER DE PROVENCE ZONE DE LA GARE, ANNOT')

0.2884615384615385

In [46]:
get_match_score('Chirurgiens Dentistes Patr',"SOCIETE D'EXERCICE LIBERALE A RESPONSABILITE LIMITEE DE CHIRURGIENS DENTISTES PATRICK SEBAG")

0.625

In [48]:
get_match_score('Taher ALi Habib', 'HABIB Taher ALi')

1.5

In [50]:
get_match_score('pl Revelly, 04240 ANNOT', 'QUA QUARTIER DE LA BEITE,4240 ANNOT')

0.3352272727272727

In [52]:
get_match_score('rte 2 Vallées, 80100 ABBEVILLE', 'RTE DES DEUX VALLEES,80100 ABBEVILLE')

0.85

In [141]:
''.join(get_overlap(doublemetaphone('Taher')[0], doublemetaphone('Taher')[0]))

'THR'

In [142]:
''.join(get_overlap(doublemetaphone('Taher')[1], doublemetaphone('Taher')[1]))

''

In [143]:
''.join(get_overlap(doublemetaphone('Taher')[0], doublemetaphone('Taher')[1]))

''

In [144]:
''.join(get_overlap(doublemetaphone('Taher')[1], doublemetaphone('Taher')[0]))

''

In [ ]:
from thefuzz import fuzz

In [38]:
fuzz.token_sort_ratio('av Gare, ANNOT','GARE DES CHEMINS DE FER DE PROVENCE ZONE DE LA GARE, ANNOT')

31

In [47]:
fuzz.token_sort_ratio('Chirurgiens Dentistes Patr',"SOCIETE D'EXERCICE LIBERALE A RESPONSABILITE LIMITEE DE CHIRURGIENS DENTISTES PATRICK SEBAG")

44

In [49]:
fuzz.token_sort_ratio('Taher ALi Habib', 'HABIB Taher ALi')

100

In [51]:
fuzz.token_sort_ratio('pl Revelly, 04240 ANNOT', 'QUA QUARTIER DE LA BEITE,4240 ANNOT')

53

In [53]:
fuzz.token_sort_ratio('rte 2 Vallées, 80100 ABBEVILLE', 'RTE DES DEUX VALLEES,80100 ABBEVILLE')

81

### Conclusions

1. _String Fuzzy matcher from the library __'thefuzz'__ is performing quite better than my own function_
2. It can match both names as well as addresses equally reliably

See here: https://github.com/seatgeek/thefuzz

# Using Recordlinkage

see here: https://recordlinkage.readthedocs.io/en/latest/about.html & https://pbpython.com/record-linking.html

### Cleaning using recordlinkage

In [19]:
from recordlinkage.preprocessing import clean

In [13]:
s1['name'] = clean(s1['name'], strip_accents='unicode')
s1['street_name'] = clean(s1['street_name'], strip_accents='unicode')
s1['street_type'] = clean(s1['street_type'], strip_accents='unicode')
s1['address_line2'] = clean(s1['address_line2'], strip_accents='unicode')
s1['city'] = clean(s1['city'], strip_accents='unicode')

/home/thabib/anaconda3/envs/nlp_siddata/lib/python3.8/site-packages/recordlinkage/preprocessing/cleaning.py:114: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(r'(\[.*?\]|\(.*?\)|\{.*?\})', '')
/home/thabib/anaconda3/envs/nlp_siddata/lib/python3.8/site-packages/recordlinkage/preprocessing/cleaning.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(replace_by_none, '')
/home/thabib/anaconda3/envs/nlp_siddata/lib/python3.8/site-packages/recordlinkage/preprocessing/cleaning.py:121: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(replace_by_whitespace, ' ')
/home/thabib/anaconda3/envs/nlp_siddata/lib/python3.8/site-packages/recordlinkage/preprocessing/cleaning.py:124: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(r'\s\s+',

In [14]:
display(s1)

,id,name,street_number,street_type,street_name,address_line2,postal_code,city
0,542012000015,societe des sucreries du marquenterre,NaN,rue,de la fontaine,NaN,80120,rue
1,542012000031,societe des sucreries du marquenterre,NaN,rte,nationale,NaN,62140,marconnelle
2,545011900028,le grand cafe et gassendi,NaN,NaN,saint christophe,route de marseille,4000,digne les bains
3,552017600016,hernas cartonnage,50.0,rue,pasteur,NaN,80210,feuquieres en vimeu
4,552017600032,hernas cartonnage,NaN,NaN,zone des hayettes,NaN,80210,chepy
...,...,...,...,...,...,...,...,...
1826164,90771119600010,bionormande,NaN,rue,de la grande flandre,lot 1 7 a,76270,neufchatel en bray
1826165,90771120400012,ahela bat,18.0,rue,saint clement,NaN,93200,saint denis
1826166,90771121200015,deco batiment,7.0,rue,auguste blanqui,NaN,93200,saint denis
1826167,90760906900014,chateaux merveilles,15.0,rue,de milan,NaN,75009,paris 9


In [15]:
s2_nond['name'] = clean(s2_nond['name'], strip_accents='unicode')
s2_nond['address'] = clean(s2_nond['address'], strip_accents='unicode')

/home/thabib/anaconda3/envs/nlp_siddata/lib/python3.8/site-packages/recordlinkage/preprocessing/cleaning.py:114: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(r'(\[.*?\]|\(.*?\)|\{.*?\})', '')
/home/thabib/anaconda3/envs/nlp_siddata/lib/python3.8/site-packages/recordlinkage/preprocessing/cleaning.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(replace_by_none, '')
/home/thabib/anaconda3/envs/nlp_siddata/lib/python3.8/site-packages/recordlinkage/preprocessing/cleaning.py:121: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(replace_by_whitespace, ' ')
/home/thabib/anaconda3/envs/nlp_siddata/lib/python3.8/site-packages/recordlinkage/preprocessing/cleaning.py:124: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(r'\s\s+',

In [16]:
display(s2_nond)

,Unnamed: 0,address,website,id,name
0,1,50 rte bage 01380 saint cyr sur menthon,NaN,84122632700010,sarl landrix freres
1,2,120 r menthon 01380 saint cyr sur menthon,NaN,79134582000025,marion kwasniewski
2,4,350 rte loeze 01380 saint cyr sur menthon,NaN,43416176600010,manigand stephane
3,5,41 r centre 01380 saint cyr sur menthon,http://www.infirmiere-frederique-thevenard.fr,48854644100012,frederique thevenard
4,6,116 chem cochatieres 01380 saint cyr sur menthon,NaN,41811188600028,garnier xavier
...,...,...,...,...,...
2271357,3530296,2 av chateau 83150 bandol,NaN,30130663500097,carlot jean francois
2271358,3530297,NaN,NaN,84978102600018,jacoby o
2271359,3530299,5 r jardins 83150 bandol,http://aquabike-en-mer.com,79518496900025,calvet richard
2271360,3530300,54 traverse laboureur 83150 bandol,http://dolcefarnientebandol.com,53925863200014,dolce far niente


In [18]:
s2_nond.to_csv('./source2_nonanids_nod_cstr.csv', index=False)

In [37]:
s2_nond[s2_nond['id'] == 84122632700010]

,Unnamed: 0,address,website,id,name
0,1,50 rte bage 01380 saint cyr sur menthon,NaN,84122632700010,sarl landrix freres


In [38]:
s1[s1['id'] == 84122632700010]

,id,name,street_number,street_type,street_name,address_line2,postal_code,city
1632255,84122632700010,landrix freres,50.0,rte,de bage,NaN,1380,saint cyr sur menthon


### Parsing 'address' in source2 into separate columns

In [18]:
from postal.parser import parse_address

# 'city' – city
# 'postcode' – psotal_code
# 'house_number' – street_number
# 'road' – street_name
# 'house' OR anything else – address_line2

In [46]:
s1[s1['street_type'] == 'cite']

,id,name,street_number,street_type,street_name,address_line2,postal_code,city
571,1286320500014,residence surcouf,195.0,cite,mal leclerc,NaN,50100,cherbourg en cotentin
1136,1485072100011,copropriete le parc,NaN,cite,du parc,NaN,15000,aurillac
2662,2101471700017,cedres,10.0,cite,epargne,NaN,83000,toulon
3101,2196242800013,copropriete la rocade,NaN,cite,de la rocade,NaN,84500,bollene
3341,3272486600010,cop res fontaine mouneyra,NaN,cite,mouneyra,NaN,33000,bordeaux
...,...,...,...,...,...,...,...,...
1822927,82145271100046,echo studio,9.0,cite,paradis,NaN,75010,paris 10
1823219,90758682000017,k corporate,NaN,cite,hlm castelsec,logement 60 esc 4,34120,pezenas
1823809,90767543300011,tanismad auto,NaN,cite,chaperon vert 3eme avenue,hlm chaperon vert pt 202 esc 12,94250,gentilly
1823810,90767544100014,les veterans de la presquile de lezardrieux se...,32.0,cite,des ajoncs dor,NaN,22610,pleubian


In [106]:
unique_street_types = np.unique(s1['street_type'].dropna())
unique_street_types

array(['all', 'av', 'bd', 'cami', 'car', 'che', 'chem', 'chs', 'cite',
       'clos', 'cor', 'cote', 'cour', 'crs', 'dom', 'dsc', 'eca', 'esp',
       'fg', 'gr', 'ham', 'hle', 'imp', 'ld', 'lot', 'mar', 'mte', 'parc',
       'pas', 'pl', 'plan', 'pln', 'plt', 'pont', 'port', 'pro', 'prv',
       'qua', 'quai', 'res', 'rle', 'roc', 'rpt', 'rte', 'rue', 'sen',
       'sq', 'tpl', 'tra', 'vla', 'vlge', 'voie', 'za', 'zac', 'zi',
       'zone'], dtype=object)

In [85]:
parse_address('22 Bis r Nationale, 45190 BEAUGENCY')

[('22 bis', 'house_number'),
 ('r nationale', 'road'),
 ('45190', 'postcode'),
 ('beaugency', 'city')]

In [114]:
txt = "h3110 bis"
[int(s) for s in txt.split() if s.isdigit()]

[]

In [116]:
re.findall(r'\d+', "22 bis")

['22']

In [87]:
parse_address('Parking Station De Lavage 505 av Georges Pompidou, 73200 GILLY SUR ISÈRE')

[('parking station de lavage', 'house'),
 ('505', 'house_number'),
 ('av georges pompidou', 'road'),
 ('73200', 'postcode'),
 ('gilly sur isère', 'city')]

In [4]:
parse_address('4 BD DE LA REPUBLIQUE,80100 ABBEVILLE')

[('4', 'house_number'),
 ('bd de la republique', 'road'),
 ('80100', 'postcode'),
 ('abbeville', 'city')]

In [5]:
parse_address('Zac De Cadréan bd Cadréan, 44550 MONTOIR DE BRETAGNE')

[('zac de cadréan', 'house'),
 ('bd cadréan', 'road'),
 ('44550', 'postcode'),
 ('montoir de bretagne', 'city')]

In [98]:
x_ = parse_address('zi Louis Delaporte, 76370 ROUXMESNIL BOUTEILLES')

In [75]:
x_ = parse_address('Zac De Cadréan bd Cadréan, 44550 MONTOIR DE BRETAGNE')

In [99]:
x_

[('zi louis delaporte', 'road'),
 ('76370', 'postcode'),
 ('rouxmesnil bouteilles', 'city')]

In [103]:
' '.join(x_[0][0].split()[1:])

'louis delaporte'

In [20]:
def create_address_fields_dataframe(full_address_series=None):
    
    df_address_fields = pd.DataFrame(columns=['street_number', 'street_type', 'street_name', 'address_line2',
                                              'postal_code', 'city'])
    
    for idx, add in enumerate(full_address_series):
        if idx%10000 == 0:
            print('{} entries completed...'.format(idx))
        str_num = str_type = str_name = addline2 = postalcode = city = np.nan
        
        if type(add) == float:
            pass
        else:
            addline2 = ''
            parsed_add = parse_address(add)
            
            for item in parsed_add:
                if item[1] == 'city':
                    city = item[0]
                elif item[1] == 'postcode':
                    postalcode = item[0]
                elif item[1] == 'road':
                    road_comps = item[0].split()
                    str_name = ' '.join(road_comps[1:])
                    if road_comps[0] == 'r':
                        str_type = 'rue'
                    elif road_comps[0] in unique_street_types:
                        str_type = road_comps[0]
                    else:
                        str_name = item[0]
                elif item[1] == 'house_number':
                    try:
                        str_num = int(item[0])
                    except ValueError:
                        #print('The parsed \'house_number\' is not a valid integer. Extracting the first integer '\
                         #     'occurrence (from string) and setting it as \'street_number\'')
                        ints = re.findall(r'\d+', item[0])
                        if len(ints) == 0:
                            str_num = np.nan
                        else:
                            sep_ints = [int(s) for s in item[0].split() if s.isdigit()]
                            if len(sep_ints) != 0:
                                str_num = sep_ints[0]
                            else:
                                str_num = ints[0]
                else:
                    addline2 += item[0]
                    addline2 += ' '
            if addline2 == '':
                addline2 = np.nan
            
        df_address_fields.loc[idx] = [str_num, str_type, str_name, addline2, postalcode, city]
        
    return df_address_fields

In [ ]:
df_address_fields = create_address_fields_dataframe(s2_nond_cstr['address'])
df_address_fields

0 entries completed...
10000 entries completed...
20000 entries completed...
30000 entries completed...
40000 entries completed...
50000 entries completed...
60000 entries completed...
70000 entries completed...
80000 entries completed...
90000 entries completed...
100000 entries completed...
110000 entries completed...
120000 entries completed...
130000 entries completed...
140000 entries completed...
150000 entries completed...
160000 entries completed...
170000 entries completed...
180000 entries completed...
190000 entries completed...
200000 entries completed...
210000 entries completed...
220000 entries completed...
230000 entries completed...
240000 entries completed...


### Matching rows using recordlinkage